<h1> <center> An introductory notebook to PyTorch module. </center ></h1>

This notebook will cover the basics of PyTorch, a popular deep learning framework. We will go through the following topics:

-  Installation of PyTorch
- Tensors in PyTorch
- Basic operations on Tensors
- Autograd: Automatic differentiation
- Building a simple neural network
- Training the neural network
- Evaluating the model
- Saving and loading models


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import time
import logging
import pandas as pd
from datetime import datetime
import math

def setup_logging():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_filename = f'1D_M_PIRL_PINN_training_{timestamp}.log'
    
    logger = logging.getLogger()
    if logger.hasHandlers():
        logger.handlers.clear()
    
    logging.basicConfig(
        level=logging.INFO,
        format='%(message)s',
        handlers=[
            logging.FileHandler(log_filename, encoding='utf-8'),
            logging.StreamHandler()
        ]
    )
    return log_filename

log_filename = setup_logging()
logger = logging.getLogger(__name__)

torch.manual_seed(42)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {DEVICE}")

if torch.cuda.is_available():
    torch.cuda.empty_cache()

def menet_pricer(t0, S0, K, T, r, sigma, num_paths, num_timesteps):
    with torch.no_grad():
        if not isinstance(r, torch.Tensor):
            r = torch.tensor(r, device=DEVICE, dtype=torch.float32)
        if not isinstance(sigma, torch.Tensor):
            sigma = torch.tensor(sigma, device=DEVICE, dtype=torch.float32)
        if not isinstance(K, torch.Tensor):
            K = torch.tensor(K, device=DEVICE, dtype=torch.float32)
        
        dt = (T - t0) / num_timesteps
        S = S0.expand(-1, num_paths)
        
        drift = (r - 0.5 * sigma**2) * dt
        vol = sigma * torch.sqrt(dt)
        
        for _ in range(num_timesteps):
            Z = torch.randn_like(S)
            S = S * torch.exp(drift + vol * Z)
        
        payoff = torch.relu(S - K)
        expected_payoff = torch.mean(payoff, dim=1, keepdim=True)
        price = expected_payoff * torch.exp(-r * (T - t0))
        return price

class ResidualPINN(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=48, output_dim=1, layers=8):
        super(ResidualPINN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_dim, hidden_dim))
        for _ in range(layers - 2):
            self.layers.append(nn.Linear(hidden_dim + input_dim, hidden_dim))
        self.layers.append(nn.Linear(hidden_dim, output_dim))
        self.activation = nn.Tanh()
        self.apply(self.init_weights)

    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            fan_in = m.in_features
            fan_out = m.out_features
            std = math.sqrt(2 / (fan_in + fan_out))
            torch.nn.init.normal_(m.weight, mean=0.0, std=std)
            m.weight.data = torch.clamp(m.weight.data, min=-2*std, max=2*std)
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x):
        original_input = x
        x = self.layers[0](x)
        x = self.activation(x)
        for i in range(1, len(self.layers) - 1):
            x_concat = torch.cat([x, original_input], dim=1)
            update = self.layers[i](x_concat)
            update = self.activation(update)
            x = x + update  
        output = self.layers[-1](x)
        return output

class StandardPINN(nn.Module):
    def __init__(self, layers=4, hidden_dim=64):
        super(StandardPINN, self).__init__()
        layer_list = [nn.Linear(2, hidden_dim), nn.Tanh()]
        for _ in range(layers - 2):
            layer_list.extend([nn.Linear(hidden_dim, hidden_dim), nn.Tanh()])
        layer_list.append(nn.Linear(hidden_dim, 1))
        self.net = nn.Sequential(*layer_list)

    def forward(self, x):
        return self.net(x)

def compute_pde_residual(model, t, S, K, r, sigma):
    V = model(torch.cat([t, S], dim=1))
    V_t = torch.autograd.grad(V, t, grad_outputs=torch.ones_like(V), create_graph=True)[0]
    V_s = torch.autograd.grad(V, S, grad_outputs=torch.ones_like(V), create_graph=True)[0]
    V_ss = torch.autograd.grad(V_s, S, grad_outputs=torch.ones_like(V_s), create_graph=True)[0]
    residual = V_t + r * S * V_s + 0.5 * sigma**2 * S**2 * V_ss - r * V
    return residual

def train_model(model_type, params):
    if model_type in ['PIRL', 'M-PIRL']:
        model = ResidualPINN(input_dim=params['input_dim'], hidden_dim=params['residual_hidden_dim'], 
                             layers=params['residual_layers']).to(DEVICE)
    elif model_type in ['PINN', 'M-PINN']:
        model = StandardPINN(layers=params['standard_layers'], hidden_dim=params['standard_hidden_dim']).to(DEVICE)
    else:
        raise ValueError(f"Unknown model_type: {model_type}")
    
    optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])
    
    if params['use_scheduler']:
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2000, gamma=0.7)

    loss_history = []
    start_time = time.time()

    logger.info(f"\n-- start training {model_type} --")
    
    for epoch in range(params['epochs']):
        t_pde = torch.rand(params['N_interior'], 1, device=DEVICE) * params['T']
        S_pde = torch.rand(params['N_interior'], 1, device=DEVICE) * \
                (params['S_max'] - params['S_min']) + params['S_min']
        t_bc = torch.ones(params['N_bc'], 1, device=DEVICE) * params['T']
        S_bc = torch.rand(params['N_bc'], 1, device=DEVICE) * \
               (params['S_max'] - params['S_min']) + params['S_min']
        
        num_batches = max(1, params['N_interior'] // params['batch_size'])
        perm_interior = torch.randperm(params['N_interior'], device=DEVICE)
        perm_bc = torch.randperm(params['N_bc'], device=DEVICE)
        
        epoch_loss = 0.0
        for b in range(num_batches):
            start_idx = b * params['batch_size']
            end_idx = min(start_idx + params['batch_size'], params['N_interior'])
            batch_size_actual = end_idx - start_idx
            idx_interior = perm_interior[start_idx:end_idx]
            
            start_bc = (b * params['batch_size']) % params['N_bc']
            idx_bc = perm_bc[start_bc : start_bc + batch_size_actual]
            if len(idx_bc) < batch_size_actual:
                extra = batch_size_actual - len(idx_bc)
                idx_bc = torch.cat([idx_bc, perm_bc[:extra]])
            
            t_pde_batch, S_pde_batch = t_pde[idx_interior], S_pde[idx_interior]
            t_bc_batch, S_bc_batch = t_bc[idx_bc], S_bc[idx_bc]
            
            optimizer.zero_grad()
            t_pde_batch.requires_grad = True
            S_pde_batch.requires_grad = True
            pde_residual = compute_pde_residual(model, t_pde_batch, S_pde_batch, 
                                               params['K'], params['r'], params['sigma'])
            loss_pde = torch.mean(pde_residual**2)

            V_pred_bc = model(torch.cat([t_bc_batch, S_bc_batch], dim=1))
            V_true_bc = torch.relu(S_bc_batch - params['K'])
            loss_bc = torch.mean((V_pred_bc - V_true_bc)**2)

            total_loss = loss_pde + params['lambda_bc'] * loss_bc

            if 'M-' in model_type:
                batch_mar = min(batch_size_actual, params['N_martingale'] // num_batches)
                t_mar = torch.rand(batch_mar, 1, device=DEVICE) * (params['T'] * 0.99)
                S_mar = torch.rand(batch_mar, 1, device=DEVICE) * \
                        (params['S_max'] - params['S_min']) + params['S_min']
                
                V_pinn_mar = model(torch.cat([t_mar, S_mar], dim=1))
                V_menet_mar = menet_pricer(t_mar, S_mar, params['K'], params['T'], params['r'], params['sigma'],
                                          params['menet_paths'], params['menet_steps'])
                loss_mar = torch.mean((V_pinn_mar - V_menet_mar)**2)
                total_loss += params['lambda_martingale'] * loss_mar

            total_loss.backward()
            optimizer.step()
            epoch_loss += total_loss.item()
        
        epoch_loss /= num_batches
        loss_history.append(epoch_loss)
        
        if params['use_scheduler']:
            scheduler.step()

        if epoch % 2000 == 0:
            elapsed = time.time() - start_time
            logger.info(f"Epoch {epoch:6d}: Loss={epoch_loss:.4e}, Time={elapsed:.2f}s")

    training_time = time.time() - start_time
    return model, loss_history, training_time

if __name__ == '__main__':
    n = 1
    shared_params = {
        'K': 1.0, 'T': 1.0, 'r': 0.05, 'sigma': 0.2,
        'S_min': 0.5, 'S_max': 1.5,
        'input_dim': 2,
        'residual_hidden_dim': 48, 'residual_layers': 8,
        'standard_hidden_dim': 64, 'standard_layers': 4,
        'lr': 1e-3, 'epochs': 10000, 'batch_size': 512,
        'use_scheduler': True,
        'lambda_bc': 100.0, 'lambda_martingale': 1.0,
        'N_interior': 10000 * n,
        'N_bc': 2000 * n,
        'N_martingale': 2000 * n,
        'menet_paths': 2000, 'menet_steps': 252
    }

    pinn_model, pinn_loss_hist, pinn_time = train_model('PINN', shared_params)
    m_pinn_model, m_pinn_loss_hist, m_pinn_time = train_model('M-PINN', shared_params)
    pirl_model, pirl_loss_hist, pirl_time = train_model('PIRL', shared_params)
    m_pirl_model, m_pirl_loss_hist, m_pirl_time = train_model('M-PIRL', shared_params)

    def black_scholes_call_numpy(t, S, K, T, r, sigma):
        epsilon = 1e-8
        T_t = T - t + epsilon
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T_t) / (sigma * np.sqrt(T_t))
        d2 = d1 - sigma * np.sqrt(T_t)
        return (S * norm.cdf(d1) - K * np.exp(-r * T_t) * norm.cdf(d2))

    t_grid_np = np.linspace(0, shared_params['T'], 100)
    S_grid_np = np.linspace(shared_params['S_min'], shared_params['S_max'], 100)
    T_mesh, S_mesh = np.meshgrid(t_grid_np, S_grid_np)
    grid_points = torch.tensor(np.stack([T_mesh.flatten(), S_mesh.flatten()], axis=-1), 
                              dtype=torch.float32).to(DEVICE)

    V_true = black_scholes_call_numpy(T_mesh, S_mesh, **{k: shared_params[k] for k in ['K', 'T', 'r', 'sigma']})

    pinn_model.eval()
    m_pinn_model.eval()
    pirl_model.eval()
    m_pirl_model.eval()
    with torch.no_grad():
        V_pred_pinn = pinn_model(grid_points).cpu().numpy().reshape(T_mesh.shape)
        V_pred_m_pinn = m_pinn_model(grid_points).cpu().numpy().reshape(T_mesh.shape)
        V_pred_pirl = pirl_model(grid_points).cpu().numpy().reshape(T_mesh.shape)
        V_pred_m_pirl = m_pirl_model(grid_points).cpu().numpy().reshape(T_mesh.shape)

    error_pinn = np.linalg.norm(V_pred_pinn - V_true) / np.linalg.norm(V_true)
    error_m_pinn = np.linalg.norm(V_pred_m_pinn - V_true) / np.linalg.norm(V_true)
    error_pirl = np.linalg.norm(V_pred_pirl - V_true) / np.linalg.norm(V_true)
    error_m_pirl = np.linalg.norm(V_pred_m_pirl - V_true) / np.linalg.norm(V_true)

    detailed_data = {
        'Time_t': T_mesh.flatten(), 'Stock_S': S_mesh.flatten(), 'V_true': V_true.flatten(),
        'V_pred_PINN': V_pred_pinn.flatten(), 'V_pred_M_PINN': V_pred_m_pinn.flatten(),
        'V_pred_PIRL': V_pred_pirl.flatten(), 'V_pred_M_PIRL': V_pred_m_pirl.flatten()
    }
    df_results = pd.DataFrame(detailed_data)
    csv_path = f"1D_M_PIRL_PINN_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df_results.to_csv(csv_path, index=False)
    logger.info(f"Data saved in: {csv_path}")

    # 1. Loss Comparison
    plt.figure(figsize=(8, 6))
    plt.plot(pinn_loss_hist, label='PINN Loss', color='green')
    plt.plot(m_pinn_loss_hist, label='M-PINN Loss', color='blue')
    plt.plot(pirl_loss_hist, label='PIRL Loss', color='red')
    plt.plot(m_pirl_loss_hist, label='M-PIRL Loss', color='purple')
    plt.yscale('log')
    plt.title('Loss Convergence Comparison')
    plt.xlabel('Epoch'); plt.ylabel('Total Loss (log scale)')
    plt.legend(); plt.grid(True, which="both", ls="--")
    plt.savefig('1_loss_convergence.png', dpi=300)
    plt.close()

    # 2. Price Curve at t=0
    plt.figure(figsize=(8, 6))
    plt.plot(S_grid_np, V_true[:, 0], 'k-', label='Analytical Solution', linewidth=2)
    plt.plot(S_grid_np, V_pred_pinn[:, 0], 'g--', label=f'PINN (Err: {error_pinn:.2%})')
    plt.plot(S_grid_np, V_pred_m_pinn[:, 0], 'b-.', label=f'M-PINN (Err: {error_m_pinn:.2%})')
    plt.plot(S_grid_np, V_pred_pirl[:, 0], 'r--', label=f'PIRL (Err: {error_pirl:.2%})')
    plt.plot(S_grid_np, V_pred_m_pirl[:, 0], 'p-.', label=f'M-PIRL (Err: {error_m_pirl:.2%})')
    plt.title('Price Curve at t=0')
    plt.xlabel('Stock Price S'); plt.ylabel('Option Price V')
    plt.legend(); plt.grid(True)
    plt.savefig('2_price_curve_t0.png', dpi=300)
    plt.close()

    logger.info("Two figures saved: 1_loss_convergence.png, 2_price_curve_t0.png")
    logger.info("Training and evaluation completed successfully!")


Using device: cpu

-- start training PINN --
Epoch      0: Loss=3.4906e+00, Time=0.33s
Epoch   2000: Loss=1.0593e-04, Time=945.85s
Epoch   4000: Loss=4.7318e-05, Time=1945.25s
Epoch   6000: Loss=3.3304e-05, Time=2884.06s
Epoch   8000: Loss=2.1530e-05, Time=3861.93s

-- start training M-PINN --
Epoch      0: Loss=2.3577e+00, Time=14.47s
Epoch   2000: Loss=1.1931e-04, Time=171136.06s


<h3> Setting PyTorch </h3>

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
torch.cuda.is_available()


False

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# -------------------------
# Device
# -------------------------
def pick_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")

device = pick_device()
print("PyTorch:", torch.__version__)
print("Device :", device)
if device.type == "cuda":
    prop = torch.cuda.get_device_properties(0)
    print("GPU    :", prop.name, f"({prop.total_memory/(1024**3):.2f} GB)")
    torch.backends.cudnn.benchmark = True

# -------------------------
# Synthetic data (no torchvision)
# -------------------------
# Tu peux augmenter N pour plus long, ou batch_size pour plus de charge GPU
N = 200_000          # nombre d'exemples
in_dim = 1024        # taille input
num_classes = 10
batch_size = 2048    # si OOM: 2048 -> 1024 -> 512
epochs = 5

# Données aléatoires
X = torch.randn(N, in_dim)
y = torch.randint(0, num_classes, (N,))
ds = TensorDataset(X, y)
loader = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=(device.type=="cuda"))

# -------------------------
# Simple FFN
# -------------------------
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, num_classes),
        )

    def forward(self, x):
        return self.net(x)

model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)

use_amp = (device.type == "cuda")
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

# -------------------------
# Train
# -------------------------
for epoch in range(1, epochs + 1):
    model.train()
    t0 = time.time()
    seen = 0
    loss_sum = 0.0

    if device.type == "cuda":
        torch.cuda.reset_peak_memory_stats()

    for step, (xb, yb) in enumerate(loader, 1):
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        if use_amp:
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                logits = model(xb)
                loss = criterion(logits, yb)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

        bs = xb.size(0)
        seen += bs
        loss_sum += loss.item() * bs

        if step % 50 == 0:
            dt = time.time() - t0
            imgs_s = seen / max(dt, 1e-9)
            msg = f"Epoch {epoch} | step {step:4d} | loss {loss_sum/seen:.4f} | {imgs_s:.0f} samples/s"
            if device.type == "cuda":
                peak = torch.cuda.max_memory_allocated() / (1024**2)
                msg += f" | peak VRAM {peak:.0f} MB"
            print(msg)

    dt = time.time() - t0
    samples_s = seen / max(dt, 1e-9)
    msg = f"\nEpoch {epoch} done | avg loss {loss_sum/seen:.4f} | throughput {samples_s:.0f} samples/s"
    if device.type == "cuda":
        peak = torch.cuda.max_memory_allocated() / (1024**2)
        msg += f" | peak VRAM {peak:.0f} MB"
    print(msg, "\n")


PyTorch: 2.9.1+cpu
Device : cpu


C:\Users\samym\AppData\Local\Temp\ipykernel_10668\1737830671.py:63: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)


Epoch 1 | step   50 | loss 2.3549 | 1117 samples/s

Epoch 1 done | avg loss 2.3294 | throughput 1065 samples/s 



<h3> Basic Operations on PyTorch </h3>

In [2]:
print("Version torch:", torch.__version__)

# Création
a = torch.tensor([1, 2, 3])                    # à partir d'une liste (dtype inféré)
b = torch.arange(0, 9).reshape(3, 3)           # arange + reshape
c = torch.randn(2, 3)                          # aléatoire normale
zeros = torch.zeros(2, 2)
ones = torch.ones(2, 2)
eye = torch.eye(3)
from_numpy = torch.from_numpy(np.array([10, 20, 30]))  # depuis numpy (même mémoire si CPU)

print("a, b, c:", a, b, c, sep="\n")

# Propriétés
print("dtype, device, shape:", c.dtype, c.device, c.shape)

# Indexing & slicing
print("b[0,1]:", b[0, 1])
print("b slice:", b[:, 1:3])

# Reshape / view / flatten / squeeze / unsqueeze
x = torch.arange(12)
print("view reshape:", x.view(3, 4).shape, x.reshape(2, 6).shape)
print("unsqueeze/squeeze:", x.unsqueeze(0).shape, x.unsqueeze(0).squeeze().shape)
print("flatten:", x.flatten().shape)

# Transpose / permute
m = torch.randn(2, 3, 4)
print("permute:", m.permute(2, 0, 1).shape)

# Concat / stack
p1 = torch.randn(2, 3)
p2 = torch.randn(2, 3)
print("cat dim0:", torch.cat([p1, p2], dim=0).shape)
print("stack dim0:", torch.stack([p1, p2], dim=0).shape)

# Arithmétique (élément-wise) et broadcasting
u = torch.tensor([1.0, 2.0, 3.0])
v = torch.tensor([[1.0], [2.0]])
print("u + 1:", u + 1)
print("broadcast add:", u + v)   # broadcasting

# Matmul / @
A = torch.randn(3, 4)
B = torch.randn(4, 2)
print("matmul:", A @ B, "shape:", (A @ B).shape)

# Reductions
t = torch.randn(5, 4)
print("sum, mean, min, max:", t.sum().item(), t.mean().item(), t.min().item(), t.max().item())
print("argmax dim0:", t.argmax(dim=0))

# Comparaisons & masque logique
mask = t > 0
print("mask sum positive:", mask.sum().item())
print("masked select:", t[t > 0])

# In-place ops (attention aux gradients)
z = torch.tensor([1.0, 2.0, 3.0])
z.add_(1.0)   # modifie z
print("in-place add_:", z)

# Clone / detach / requires_grad / backward
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = (x ** 2).sum()
y.backward()            # dy/dx = 2*x
print("grad:", x.grad)

xd = x.detach().clone() # détaché du graphe

# Conversion vers numpy (doit être sur CPU)
arr = x.cpu().detach().numpy()
print("numpy:", arr, type(arr))

# Déplacement device / dtype conversion
if torch.cuda.is_available():
    print("CUDA disponible, envoi sur GPU")
    A_cuda = A.to('cuda')
    A_cpu = A_cuda.to('cpu')

print("astype (float->int):", torch.tensor([1.2, 2.8]).to(torch.int))

# Utilitaires: clamp, round, sort, unique
vals = torch.tensor([-1.5, 0.2, 3.7, 3.7])
print("clamp:", vals.clamp(0, 3))
print("round:", vals.round())
print("unique:", vals.unique())
print("sort:", vals.sort().values)

# Autres utiles: einsum, torch.nn.functional ops
print("einsum (trace):", torch.einsum('ii->', torch.randn(3, 3)))

# Remise à zéro des gradients
x.grad.zero_()

# Exemple rapide résumé
print("Exemple résumé OK")
# ...existing code...

Version torch: 2.9.1+cpu
a, b, c:
tensor([1, 2, 3])
tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
tensor([[-0.8073,  0.5935,  0.9673],
        [-0.1903,  0.2044, -0.1103]])
dtype, device, shape: torch.float32 cpu torch.Size([2, 3])
b[0,1]: tensor(1)
b slice: tensor([[1, 2],
        [4, 5],
        [7, 8]])
view reshape: torch.Size([3, 4]) torch.Size([2, 6])
unsqueeze/squeeze: torch.Size([1, 12]) torch.Size([12])
flatten: torch.Size([12])
permute: torch.Size([4, 2, 3])
cat dim0: torch.Size([4, 3])
stack dim0: torch.Size([2, 2, 3])
u + 1: tensor([2., 3., 4.])
broadcast add: tensor([[2., 3., 4.],
        [3., 4., 5.]])
matmul: tensor([[ 0.4739,  0.4555],
        [-0.9184, -2.1857],
        [-0.5198,  3.8290]]) shape: torch.Size([3, 2])
sum, mean, min, max: -3.723116636276245 -0.18615582585334778 -2.5148234367370605 1.3412643671035767
argmax dim0: tensor([2, 1, 2, 0])
mask sum positive: 9
masked select: tensor([0.4465, 0.5806, 1.2030, 0.7499, 0.1512, 1.3413, 0.2629, 0.0161, 0.9

<h1> <center> A comparison with Numpy :  </center> </h1>

Some exercises to train :